# 多自由度系の固有値解析

吉田勝俊（宇都宮大学）

## 参考情報
- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
- [【Python】行列指数関数・行列対数関数 - Qiita](https://qiita.com/Mrrmm252/items/a50a9b352e5064e40cc1)

In [ ]:
%matplotlib inline

import numpy as np                 #数値計算ライブラリ
from scipy.integrate import odeint #常微分方程式ライブラリ
import scipy.linalg as la          #線形代数ライブラリ
import matplotlib.pyplot as plt    #描画ライブラリ
plt_config = {
    'text.usetex': True,
    'text.latex.preamble': r"\usepackage{amsmath}",
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'lines.linewidth': 1.2,
    'lines.markersize': 3,
    'lines.markeredgewidth': 0.7,
    'lines.markerfacecolor': 'white',
    'lines.markeredgecolor': 'black',
}
plt.rcParams.update(plt_config)

## ◯行列指数関数

### ■指数関数

In [ ]:
np.exp(2.5) #普通の指数関数

### ■行列指数関数

In [ ]:
A = np.array([
    [0, 1],
    [-3, -2]
])
display(A)

#### （誤）Numpy の `exp(行列)` $\neq$ 行列指数関数

In [ ]:
np.exp(A) #これは単なる各成分の指数関数値

In [ ]:
for i in range(2):
    for j in range(2):
        print(np.exp(A[i,j])) 

#### <font color="red">（正）Scipy の `expm(行列)` $=$ 行列指数関数</font>．

In [ ]:
la.expm(A)

## ◯解の表示（多次元）

In [ ]:
def Simulation(A, x0, time):
    '''
    線形状態方程式 dx/dt = Ax を解く
    by 有限差分法による数値シミュレーション
    '''
    def eom(x, t):
        return A.dot(x)
        
    motion = odeint(
        eom,   #運動方程式を表すユーザ関数
        x0,    #初期条件
        time   #時間軸を表す数列
    )
  
    return motion

def Exp_tA_x0(A, x0, time):
    '''
    線形状態方程式 dx/dt = Ax の解を計算する
    by 行列指数関数による表示 x(t)= exp(tA)x0
    '''
    motion = [] #空のリスト
    for t in time:
        motion.append( #各時刻の解をリストに追加
            la.expm(t*A).dot(x0)
        )
  
    return np.array(motion) #Numpy配列に変換して返す

def plot_sim_vs_exp(A, x0, tminmax=[0,30], tn=200):
    '''
    数値シミュレーション vs 行列指数関数による解のプロット
    '''
    x0 = np.array(x0) #初期値
    ts = np.linspace(*tminmax, tn) #時間軸
    
    xs_sim = Simulation(A, x0, ts)
    xs_exp = Exp_tA_x0(A, x0, ts)
    
    fig, ax = plt.subplots(2,1,figsize=(6,4))

    for i in range(2):
        ax[i].plot(ts, xs_sim[:,i], 'o', 
                label=r'Simulation')
        ax[i].plot(ts, xs_exp[:,i], '-', 
                label=r'$e^{tA}\boldsymbol{x}_0$')
        ax[i].legend()
        ax[i].set_xlabel(r'$t$')
        ax[i].set_ylabel(r'$x_%d$'%(i+1))

### 演習 9.1 ( 線形振動系の解の表示 )

- 状態方程式: $\displaystyle
\dot{\boldsymbol{x}} = A\boldsymbol{x}
,\quad
\boldsymbol{x}(0)=\boldsymbol{x}_0
,\quad
A:=\begin{bmatrix}
0 & 1\\
-k/m & -c/m
\end{bmatrix}
$
- 行列指数関数による解の表示: $\boldsymbol{x}(t)=e^{tA}\boldsymbol{x}_0$

In [ ]:
def A_L1DOF(param):
    '''
    線形1自由度（linear 1-degree-of-freedom）
    の振動系を表す行列
    '''
    m, c, k = param  #パラメータの成分
    A = np.array([
        [0, 1],
        [-k/m, -c/m],
    ])
    
    return A

In [ ]:
param, x0 = [1, 0.2, 2], [1,0]

plot_sim_vs_exp(A_L1DOF(param), x0)

In [ ]:
param, x0 = [1, 2, 1], [1,2] #ちなみに固有値が重根の場合

plot_sim_vs_exp(A_L1DOF(param), x0)

#### 比較結果

- シミュレーションと $e^{tA}\boldsymbol{x}_0$ の結果は，パラメータや初期値を変えても一致します！

## ◯固有値と固有ベクトル

### 演習 9.3 ( 振動と行列の固有値の数値計算 )

#### $s^2 + 3s + 2 = 0$ の根

In [ ]:
np.roots([1, 3, 2])

#### 行列 $\begin{bmatrix}0&1\\-2&-3\end{bmatrix}$ の固有値

In [ ]:
B = np.array([
    [ 0,  1],
    [-2, -3]
])

固有値 $s_i$ と固有ベクトル $\boldsymbol{v}_i$ を求めます
- `ss`: $\boldsymbol{s}:=[s_1,\cdots,s_n]$ 固有値を並べた配列 
- `U`: $U:=[\boldsymbol{u}_1,\cdots,\boldsymbol{u}_n]$ 単位固有ベクトルを並べた行列

In [ ]:
ss, U = la.eig(B)

In [ ]:
print(ss)

固有方程式と同じ固有値が得られました．

- <font color='red'>2次方程式の根と順番が違いますが，これは単に`roots`と`eig`の仕様（結果の並べ方）の違いです．</font>
- 行列の固有値には虚部 `0.j` = 0 が見えてますが，これも単なる仕様の問題です．

固有値とその単位固有ベクトルを並べて表示してみます．

In [ ]:
for i, s in enumerate(ss):
    u = U[:,i] #固有ベクトル＝計算結果の列ベクトル
    print( 's =', s, ': u =', u)

例題9.3の手計算の結果 $s=-1:\boldsymbol{v}=(1,-1)^T$，$s=-2:\boldsymbol{v}=(1,-2)^T$ と比較すると，
- 同じ固有値が得られている．
- スカラ倍の違いを除き，同じ固有ベクトルが得られている．

## ◯複素共役による実数化

### 演習 9.4 ( 初期値の展開の数値計算 )

In [ ]:
A = np.array([ #固有値が複素数になるような行列
    [0, 1],
    [-1, -1]
])

#### 固有値と単位固有ベクトル

In [ ]:
ss, U = la.eig(A)

us = []
for i in range(len(ss)):
    us.append(U[:,i])

us = np.array(us)

for i in range(len(ss)):
    print(ss[i], us[i])

- 固有値・固有ベクトルが，確かに，共役複素数のペアで得られている

#### 初期値の展開係数

In [ ]:
x0 = np.array([5, 6]) #適当な初期値

In [ ]:
etas = la.inv(U).dot(x0) #初期値の展開係数
print(etas)

- 展開係数も，共役複素数のペアになっている

#### 初期値の復元（実数化）

In [ ]:
dim = len(ss) #次元

x0_rec = np.zeros(dim) #ゼロベクトル
for i in range(len(ss)): #展開係数*固有ベクトル の線形結合
    x0_rec = x0_rec + etas[i]*us[i]  

print(x0_rec)

- 虚部の計算機誤差$\approx -8\times 10^{-16}$ を除けば，元の実数ベクトル `[5,6]` が復元されている．

## ◯固有値によるダイナミクスの分類

In [ ]:
def plot_multidim(ts, xs):
    '''
    多次元の解をプロットする
    '''
    tn, dim = np.shape(xs)

    fig, ax = plt.subplots(1,1,figsize=(6,2))

    ax.plot(ts, xs, '-')
    ax.set_xlabel(r'$t$')
    ax.set_ylabel(r'$x_i$')

    labels = [r'$x_%d$'%(i+1) for i in range(dim)]
    ax.legend(labels=labels)
    ax.grid()

### 演習 9.5 ( モード展開の数値計算 )

#### 固有値と単位固有ベクトル

In [ ]:
A = np.array([
    [0, 1, 0],
    [0, 0, 1],
    [-1, -1, -1],
])

ss, U = la.eig(A)
us = [U[:,i] for i in range(len(ss))]
print(ss)

- 負の実根と，虚根の共役なペアが得られました．

#### 初期値の展開係数

In [ ]:
x0 = np.array([1, -1, 2]) #適当な初期値
etas = la.inv(U).dot(x0) #初期値の展開係数
print(etas)

#### 時間軸

In [ ]:
ts = np.linspace(0,20,200)

#### 「（a）実根」の成分

In [ ]:
ss[0]

In [ ]:
xs_a = np.array([
    etas[0]*np.exp(ss[0]*t)*us[0]
    for t in ts
])

xs_a = np.real(xs_a) #虚部 ≒ 0 の計算技誤差を除去
plot_multidim(ts, xs_a)

- 負の実根に対応する，非振動減衰が見て取れます．

#### 「（ｂ）共役な虚根」の成分

In [ ]:
ss[1], ss[2]

In [ ]:
xs_b = np.array([
    etas[1]*np.exp(ss[1]*t)*us[1] + etas[2]*np.exp(ss[2]*t)*us[2]
    for t in ts
])

xs_b = np.real(xs_b) #虚部 ≒ 0 の計算技誤差を除去
plot_multidim(ts, xs_b)

- 共役な虚根に対応する単振動が見て取れます．

#### 「（ｄ）異なるダイナミクスの総和」

In [ ]:
ss[0], ss[1], ss[2]

In [ ]:
dim = len(ss)

xs_d = xs_a + xs_b #全ての総和

xs_d = np.real(xs_d) #虚部 ≒ 0 の計算技誤差を除去
plot_multidim(ts, xs_d)

- $\boldsymbol{x}(t)=$ （負の実根の成分）+（共役複素根の成分）のプロットです．